# <font color=#0404B4>**SEMINAR - PHÂN TÍCH DỮ LIỆU THÔNG MINH** </font>

### <font color=#0404B4> Bảng thành viên nhóm </font>
|<center><div style="width:150px">MSSV</div><center>|<center><div style="width:290px">Họ và tên</div><center>|
|---------- |:-------------:|
| <center>20120032<center>  | <center>Phan Trường An<center> |
| <center>20120037<center>  | <center>Trần Thị Minh Anh<center> |
| <center>20120041<center>  | <center>Trần Kim Bảo<center> |
| <center>20120053<center>  |   <center>Nguyễn Thành Đạt<center>   | 
| <center>20120071<center>  |<center>Nguyễn Thị Bích Hà<center> |
| <center>20120095<center>  | <center>Ngô Quang Hưng<center> |

# <font color=#0404B4>Import thư viện</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

<a name="2"></a>
# <font color=#0404B4> A. Tiền xử lý dữ liệu</font>


### 1. Đọc dữ liệu

In [2]:
item_category_df=pd.read_csv("item_categories.csv")
item_category_df.shape

(84, 2)

In [3]:
items_df=pd.read_csv("items.csv")
items_df.shape

(22170, 3)

In [4]:
sales_train_df=pd.read_csv("sales_train.csv")
sales_train_df.shape

(2935849, 6)

In [5]:
sample_submission_df=pd.read_csv("sample_submission.csv")
sample_submission_df.shape

(214200, 2)

In [6]:
shops_df=pd.read_csv("shops.csv")
shops_df.shape

(60, 2)

In [7]:
test_df=pd.read_csv("test.csv")
test_df.shape

(214200, 3)

### 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- `item_category_df`: Mỗi dòng là thông tin về danh mục sản phẩm gồm: tên danh mục sản phẩm và id của danh mục sản phẩm đó.
- `items_df`: Mỗi dòng là thông tin về sản phẩm gồm: tên sản phẩm, id của sản phẩm và id của danh mục chứa sản phẩm đó.
- `sales_train_df`: Mỗi dòng là thông tin về bán hàng gồm: ngày bán hàng, thuộc kì bán nào(1 kì ở đây tương ứng với 1 tháng), id của cửa hàng bán sản phẩm đó, id của sản phẩm bán, thông tin về giá của sản phẩm và số lượng bán sản phẩm đó trong 1 ngày.
- `sample_submission_df`: Hướng dẫn định dạng nộp bài gồm các thông tin: ID(thứ tự) dự đoán, số lượng sản phẩm bán ra trong tháng.
- `shops_df`: Mỗi dòng là thông tin về cửa hàng gồm: tên cửa hàng và id của cửa hàng đó
- `test_df`: Chứa dữ liệu kiểm tra cho mô hình dự đoán gồm ID dự đoán, id cửa hàng và id sản phẩm.

### 3. Có dữ liệu bị trùng lặp ở các dataframe không

In [8]:
def check_duplicated(dataframe):
    duplicated_rows = dataframe.duplicated()

    # In các hàng bị lặp lại
    if len(dataframe[duplicated_rows])>0:
        print('Có dòng dữ liệu bị lặp')
        display(dataframe[duplicated_rows])
    else: 
        print('Không có dòng dữ liệu nào bị lặp')


Dataframe `item_category_df`

In [9]:
check_duplicated(item_category_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `items_df`

In [10]:
check_duplicated(items_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `sales_train_df`

In [11]:
check_duplicated(sales_train_df)

Có dòng dữ liệu bị lặp


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
76962,05.01.2013,0,54,20130,149.0,1.0
1435367,23.02.2014,13,50,3423,999.0,1.0
1496766,23.03.2014,14,21,3423,999.0,1.0
1671873,01.05.2014,16,50,3423,999.0,1.0
1866340,12.07.2014,18,25,3423,999.0,1.0
2198566,31.12.2014,23,42,21619,499.0,1.0


Xóa luôn các mẫu dữ liệu bị trùng

In [12]:
sales_train_df.drop_duplicates(inplace=True)

Dataframe `sample_submission_df`

In [13]:
check_duplicated(sample_submission_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `shops_df`

In [14]:
check_duplicated(shops_df)

Không có dòng dữ liệu nào bị lặp


Dataframe `test_df`

In [15]:
check_duplicated(test_df)

Không có dòng dữ liệu nào bị lặp


<h3> 4. Mỗi cột có ý nghĩa gì?</h3>

|STT|Tên cột dữ liệu | <center>Mô tả<center> | Đơn vị |
|--|:------:| ---| --- |
|1|item_category_name|<center>Tên danh mục sản phẩm<center>|
|2|item_category_id| <center>Mã định danh của danh mục sản phẩm<center>|
|3|item_name|<center>Tên sản phẩm<center>| 
|4|item_id| <center>Mã định danh sản phẩm<center>||
|5|date|<center>Ngày bán hàng<center>|| 
|6|date_block_num|<center>Thứ tự kỳ bán hàng *(bắt đầu từ 0)*<center>|
|7|shop_id|<center>Mã định danh cửa hàng<center>| |
|8|item_price|<center>Giá sản phẩm<center>|<center>Rúp<center>|
|9|item_cnt_day|<center>Số lượng bán ra/ số lượng mất trong `1 ngày`<center>|<center>Sản phẩm <center>|
|10|ID|<center>Mã định danh<center>| |
|11|item_cnt_month|<center>Số lượng bán ra/ số lượng mất trong `1 tháng`<center>|<center>Sản phẩm <center>|
|12|shop_name|<center>Tên cửa hàng<center>| |


### 5. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp?



Dataframe `item_category_df`

In [16]:
item_category_df.columns

Index(['item_category_name', 'item_category_id'], dtype='object')

In [17]:
item_category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


Dataframe `items_df`

In [18]:
items_df.columns

Index(['item_name', 'item_id', 'item_category_id'], dtype='object')

In [19]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


Dataframe `sales_train_df`

In [20]:
sales_train_df.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')

In [21]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935843 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 156.8+ MB


Dataframe `sample_submission_df`

In [22]:
sample_submission_df.columns

Index(['ID', 'item_cnt_month'], dtype='object')

In [23]:
sample_submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              214200 non-null  int64  
 1   item_cnt_month  214200 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 3.3 MB


Dataframe `shops_df`

In [24]:
shops_df.columns

Index(['shop_name', 'shop_id'], dtype='object')

In [25]:
shops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


Dataframe `test_df`

In [26]:
test_df.columns

Index(['ID', 'shop_id', 'item_id'], dtype='object')

In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB



##### Cột `date` của dataframe `sales_train_df` đang có kiểu dữ liệu `object`. Thực hiện xử lý đưa về đúng kiểu dữ liệu `datime64[ns]`.

In [28]:
sales_train_df.date=pd.to_datetime(sales_train_df.date,format="%d.%m.%Y")

In [29]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935843 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 156.8 MB



### 7. Với mỗi cột, các giá trị được phân bố như thế nào?

- Nhận xét:
    - Các dataframe như: `item_category_df`, `items_df`, `sample_submission_df`, `shops_df` và `test_df`, là những dataframe chứa thông tin chi tiết đối đối với dataframe `sales_train_df`. Nên sẽ chỉ tập trung xem phân bố dữ liệu của dataframe `sales_train_df`
    - Với các dữ liệu trong dataframe `sales_train_df` chỉ có dạng số.

#### Cột có kiểu dữ liệu dạng số, và phân bố:


In [30]:
numerical_cols = sales_train_df[sales_train_df.select_dtypes(exclude=['object']).columns]

def missing_ratio(c):
    return  c.isna().sum()/len(c) * 100

def min(c):
    return c.min()

def max(c):
    return c.max()

nume_col_profiles_df = numerical_cols.agg([missing_ratio, min, max])
nume_col_profiles_df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0
min,2013-01-01 00:00:00,0.0,0.0,0.0,-1.0,-22.0
max,2015-10-31 00:00:00,33.0,59.0,22169.0,307980.0,2169.0


- Nhận xét:
    - `date`: dữ liệu bán hàng được thu thập từ ngày 01-01-2013 đến ngày 31-10-2015.
    - `date_block_num`: Chia dữ liệu thành 34 kì (1 kì=1 tháng)
    - `shop_id`: ID cửa hàng có giá trị nhỏ nhất là 0, lớn nhất là 59.
    - `item_id`: ID sản phẩm có giá trị nhỏ nhất là 0, lớn nhất là 22169.
    - `item_price`: Sản phẩm có giá tiền nhỏ nhất -1, lớn nhất là 307980 (Rúp).
    - `item_cnt_day`: Số lượng sản phẩm đó được bán ra. Số lượng sản phẩm bị mất lớn nhất là 22 sản phẩm, có thể nhiều lý do như hư hỏng, thất lạc trong quá trình vận chuyển,... Số lượng sản phẩm được bán ra lớn nhất là 2169 sản phẩm. 

###  8. Có cột nào cần phải tiền xử lý hay không?


Kiểm tra thử số lượng `item_price` có giá bị âm

In [31]:
sales_train_df[sales_train_df.item_price<0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,2013-05-15,4,32,2973,-1.0,1.0


Với chỉ 1 giá sản phẩm bị âm nên ta có thể thực hiện loại xóa dữ liệu đó.

In [32]:
index_drop=sales_train_df[sales_train_df.item_price<0].index
sales_train_df.drop(index_drop,inplace=True)

<a name="2"></a>
# <font color=#0404B4> B. Khám phá dữ liệu</font>


<a name="2"></a>
# <font color=#0404B4> C. Xây dựng mô hình dự đoán</font>


### `Model Long Short-Term Memory`

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import  tensorflow.keras.optimizers as optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#### Tiền xử lý để đưa dữ liệu vào mô hình

In [33]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935842 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 156.8 MB


Giảm kích thước dữ liệu giúp giảm bớt bộ nhớ lưu trữ

In [34]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df
train_df_lstm=sales_train_df.__deepcopy__()
train_df_lstm = downcast_dtypes(train_df_lstm)
train_df_lstm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935842 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int16         
 2   shop_id         int16         
 3   item_id         int16         
 4   item_price      float32       
 5   item_cnt_day    float32       
dtypes: datetime64[ns](1), float32(2), int16(3)
memory usage: 84.0 MB


Với thuộc tính `item_price` không sử dụng trong mô hình nên thực hiện loại bỏ bớt

In [35]:
train_df_lstm.drop(columns='item_price',inplace=True)

Tính tổng số lượng hàng bán được của mỗi sản phẩm của 1 cửa hàng theo tháng (có 1 số trường hợp sản phẩm được bán ở nhiều cửa hàng)

In [36]:
train_df_lstm=train_df_lstm.groupby(["date_block_num","shop_id","item_id"]).sum("item_cnt_day").rename(columns={"item_cnt_day":"item_cnt_month"}).reset_index().copy()

Chuyển dataframe thành: các cột lần lượt là `shop_id`,`item_id`, và số lượng sản phẩm bán được theo từng tháng(kì) 

In [37]:
train_df_lstm=train_df_lstm.pivot_table(columns=["date_block_num"],index=["item_id","shop_id"],values=["item_cnt_month"],fill_value=0)
train_df_lstm.reset_index(inplace=True)

merge dataframe vừa được tạo với dataframe `test_df` tạo thành dataframe `train_df_lstm`

In [38]:
train_df_lstm=pd.merge(test_df,train_df_lstm,on = ['item_id','shop_id'],how = 'left')
train_df_lstm.fillna(0,inplace=True)

/tmp/ipykernel_5548/3407239410.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  train_df_lstm=pd.merge(test_df,train_df_lstm,on = ['item_id','shop_id'],how = 'left')
/tmp/ipykernel_5548/3407239410.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  train_df_lstm=pd.merge(test_df,train_df_lstm,on = ['item_id','shop_id'],how = 'left')


In [39]:
train_df_lstm

,ID,shop_id,item_id,"(item_cnt_month, 0)","(item_cnt_month, 1)","(item_cnt_month, 2)","(item_cnt_month, 3)","(item_cnt_month, 4)","(item_cnt_month, 5)","(item_cnt_month, 6)",...,"(item_cnt_month, 24)","(item_cnt_month, 25)","(item_cnt_month, 26)","(item_cnt_month, 27)","(item_cnt_month, 28)","(item_cnt_month, 29)","(item_cnt_month, 30)","(item_cnt_month, 31)","(item_cnt_month, 32)","(item_cnt_month, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Thực hiện loại bỏ bớt các thuộc tính không dùng trong quá trình huấn luyện mô hình: `ID`,`shop_id`,`item_id`. Và đổi tên các số lượng bán hàng theo từng kì để tiện cho việc sử dụng sau này

In [40]:
train_df_lstm.drop(['ID','shop_id','item_id'],inplace = True, axis = 1)

In [41]:
train_df_lstm.columns=np.arange(0,34)

In [123]:
train_df_lstm.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Lần lượt tạo các tập dữ liệu `train`, `validation` và `test` cho mô hình

In [74]:
# Lấy data train
X_train = np.expand_dims(train_df_lstm.values[:,:-2],axis = 1)
y_train = train_df_lstm.values[:,-2]
# Lấy data validation
X_val = np.expand_dims(train_df_lstm.values[:,1:-1],axis = 1)
y_val = train_df_lstm.values[:,-1]
# Lấy dữ liệu test
X_test = np.expand_dims(train_df_lstm.values[:,2:],axis = 1)
 
print(X_train.shape, X_val.shape, X_test.shape)

(214200, 1, 32) (214200, 1, 32) (214200, 1, 32)


#### Định nghĩa các lớp cho mô hình `LTSM`:

In [76]:
model=Sequential()
model.add(LSTM(64, activation= 'relu', return_sequences=True,  input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(64, activation= 'relu',return_sequences=False))
model.add(Dense(1))
model.compile(optimizer=optimizers.Adam(lr=.0001), loss= 'mse', metrics = ['mean_squared_error'])    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1, 64)             24832     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 57,921
Trainable params: 57,921
Non-trainable params: 0
_________________________________________________________________


Sử dụng `callbacks` tối ưu hóa mô hình:
- `EarlyStopping`: Giảm thiểu overfitting trong quá trình huấn luyện, với:
    - `patience=5` : Không có sự cải thiện `loss` sau patience lặp, thực hiện dừng quá trình huấn luyện lại.
    - `verbose=1` : In ra thông tin về quá trình dừng sớm.

- `ReduceLROnPlateau`: Giảm learning rate nếu kết quả của mô hình không cải thiện, với:
    -`factor =0.25, patience=2`: sau 2 lần lặp không có sự cải thiện `loss`. Giảm learning rate xuống 1/4 so với learning rate hiện tại.
    - `min_lr=0.00001`: giá trị nhỏ nhất của learning rate.
    - `verbose=1` : In ra thông tin về việc giảm learning rate.

- `ModelCheckpoint`: Lưu lại trọng số (weights) của mô hình:
    - `model.h5`: Tên file lưu.
    - `save_best_only=True`: chỉ lưu lại trọng số của mô hình nếu kết quả đạt được là tốt nhất. 
    - `save_weights_only=True`: chỉ lưu lại trọng số của mô hình chứ không lưu cả kiến trúc của mô hình.

In [77]:
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ReduceLROnPlateau(factor=0.25, patience=2, min_lr=0.000001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [80]:
model.fit(X_train, y_train, epochs=15,batch_size=32, callbacks=callbacks, validation_data=(X_val, y_val))

Epoch 1/15


2023-05-19 18:27:57.715854: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27417600 exceeds 10% of free system memory.


6680/6694 [============================>.] - ETA: 0s - loss: 2.9585 - mean_squared_error: 2.9585

2023-05-19 18:28:09.903983: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27417600 exceeds 10% of free system memory.



Epoch 1: val_loss improved from inf to 25.30194, saving model to model.h5
6694/6694 [==============================] - 17s 2ms/step - loss: 2.9548 - mean_squared_error: 2.9548 - val_loss: 25.3019 - val_mean_squared_error: 25.3019 - lr: 0.0010
Epoch 2/15
6660/6694 [============================>.] - ETA: 0s - loss: 2.8105 - mean_squared_error: 2.8105
Epoch 2: val_loss improved from 25.30194 to 23.94525, saving model to model.h5
6694/6694 [==============================] - 14s 2ms/step - loss: 2.8012 - mean_squared_error: 2.8012 - val_loss: 23.9452 - val_mean_squared_error: 23.9452 - lr: 0.0010
Epoch 3/15
6680/6694 [============================>.] - ETA: 0s - loss: 2.2505 - mean_squared_error: 2.2505
Epoch 3: val_loss did not improve from 23.94525
6694/6694 [==============================] - 14s 2ms/step - loss: 2.2497 - mean_squared_error: 2.2497 - val_loss: 27.4728 - val_mean_squared_error: 27.4728 - lr: 0.0010
Epoch 4/15
6687/6694 [============================>.] - ETA: 0s - loss: 4.9

Dự đoán kết quả và lưu vào file

In [81]:
submit=model.predict(X_test)

   1/6694 [..............................] - ETA: 19:02

2023-05-19 18:29:48.499969: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27417600 exceeds 10% of free system memory.


6694/6694 [==============================] - 5s 766us/step


In [83]:
submit=submit.clip(0,20)
submission_df=pd.DataFrame({"ID":test_df.ID,"item_cnt_month":submit.ravel()})

In [121]:
submission_df.to_csv("submission.csv",index=False)